In [1]:
import pandas as pd
import json
import requests

In [2]:
response = requests.get('https://docs.google.com/spreadsheets/d/1TaFZwPyKXRcLBOmhbpRBFD2f2N3u5cOeuViyQkkxZ3A/export?format=csv')
with open('spreadsheet.csv', 'wb') as f:
    f.write(response.content)
df_links = pd.read_csv('spreadsheet.csv', usecols=['cardId', 'Name', 'cardSubtitle', 'cardFilter4Tri',
       'cardFilter4Display', 'Crédit image', 'cardFilter0Tri', 'cardFilter5Display', 'cardSection5Text',
'cardSection6Text', 'New'])

In [3]:
df_links['isNew'] = df_links['New'].apply(lambda x: 'is-new' if x == 'New' else 'not-new')

In [4]:
df_stories = pd.read_csv('stories.csv', usecols=['Text', 'parameter'])
df_stories['cardId'] = df_stories.index + 1

In [5]:
df = pd.read_csv('spreadsheet-after-work.csv', usecols=['cardId', 'Image', 'New',
       'Age', 'cardFilter1Tri', 'cardFilter2Tri', 'cardFilter3Tri',
       'Image_URL'])

In [6]:
df['cardImage'] = df['Image']

In [7]:
df_links = df_links[df_links['Name'] == df_links['Name']].copy()
df_links['cardId'] = df_links['cardId'].astype(int)

In [8]:
df = df.merge(df_links, on='cardId', how='outer')

In [9]:
df = df.merge(df_stories, on='cardId', how='left')
df['Text'].fillna('', inplace=True)

In [10]:
def cutText(text):
    if text.count('\n') > 2:
        p = text.split('\n')
        return "\n".join(p[:2]).replace('\n', '</p><p>') + ' <a class="readMore" href="#more">Plus…</a>'
    else:
        return text

def getCollapsedText(text):
    if text.count('\n') > 2:
        p = text.split('\n')
        return "\n".join(p[2:])
    else:
        return ''
    
df['cardSection1Text'] = df['Text'].apply(cutText)

df['cardSection2Text'] = df['Text'].apply(getCollapsedText)
df['cardInfo1'] = df['cardSubtitle']
df['cardInfo2'] = df['Age']

In [11]:
data = json.load(open('data.json'))

In [12]:
columns = list(data['cards'][0].keys())
columns.append('parameter')
columns.append('cardSection2Text')
columns.append('cardFilter0Tri')
columns.append('isNew')

In [13]:
df['cardTitre'] = df['Name']

In [14]:
dfm = df.copy()
for col in dfm.columns:
    if col not in columns:
        del dfm[col]

In [15]:
# dfm['cardImage'] = df['cardId'].apply(lambda x: '2018/bados.jpg' if x%2 == 0 else '2018/warnking.jpg')

## Filter 1: catégorie (indiquée sous le titre)

In [16]:
#dfm['cardFilter1Display'] = #'Les précurseurs'
#dfm['cardFilter1Tri'] = 'f-precurseurs'

# Filter 2: l’âge

In [17]:
# dfm['cardFilter2Tri'] = dfm['cardFilter2Tri'].apply(lambda x: x.replace('20', '40') if x == x else x)

In [18]:
dfm['cardFilter2Tri']

0      f-alemanique
1          f-romand
2      f-alemanique
3      f-alemanique
4          f-romand
5      f-alemanique
6          f-romand
7       f-italienne
8          f-romand
9          f-romand
10            expat
11            expat
12     f-alemanique
13     f-alemanique
14         f-romand
15         f-romand
16         f-romand
17     f-alemanique
18         f-romand
19     f-alemanique
20     f-alemanique
21     f-alemanique
22     f-alemanique
23     f-alemanique
24     f-alemanique
25     f-alemanique
26     f-alemanique
27            expat
28      f-italienne
29     f-alemanique
           ...     
72         f-romand
73            expat
74     f-alemanique
75     f-alemanique
76     f-alemanique
77     f-alemanique
78     f-alemanique
79            expat
80     f-alemanique
81     f-alemanique
82         f-romand
83     f-alemanique
84     f-alemanique
85         f-romand
86         f-romand
87     f-alemanique
88         f-romand
89     f-alemanique
90     f-alemanique


## Filter 3: le sexe

In [19]:
# mieux utiliser sex machine

## Filter 4: la région

In [20]:
#dfm['cardFilter4Tri'] = df['cardId'].apply(lambda x: 'f-romand' if x%3 == 0 else 'f-alemanique')

In [21]:
def getStoryLink(cell):
    try:
        title, anchor = cell.split('|')
    except:
        return ''
    return anchor

def getStoryTitle(cell):
    try:
        title, anchor = cell.split('|')
    except:
        return ''
    return title
#def getLinkedinLink(cell):
#    if cell == cell:
#        return ''

In [22]:
dfm['cardSection5Text'].value_counts()

«Un millionnaire de la Silicon Valley pour doper l’entrepreneuriat en Suisse» | https://www.letemps.ch/economie/un-millionnaire-silicon-valley-doper-lentrepreneuriat-suisse              1
«Le fondateur de Sophia Genetics nommé entrepreneur de l'année» | https://www.letemps.ch/economie/fondateur-sophia-genetics-nomme-entrepreneur-lannee                                     1
«Apple pourrait ainsi devenir un acteur majeur en Suisse»|https://www.letemps.ch/economie/apple-pourrait-ainsi-devenir-un-acteur-majeur-suisse                                            1
«La chasse aux meilleurs prix comme modèle d’affaires» | https://www.letemps.ch/economie/chasse-aux-meilleurs-prix-modele-daffaires                                                       1
«Leila Delarive et Fred Valet, les architectes de Be Curious» | https://www.letemps.ch/suisse/leila-delarive-fred-valet-architectes-be-curious                                            1
«Il y a de l’innovation en Suisse, mais peu d’entrepreneurs 

In [23]:
#dfm['cardSection5Text'] = '<a href="https://www.linkedin.com/in/marcoabele" target="_blank">Son profil LinkedIn</a>'
dfm['cardSection4Text'] = dfm['cardSection5Text'].apply(getStoryTitle)
dfm['cardSection5Text'] = dfm['cardSection5Text'].apply(getStoryLink)

In [24]:
emptyColumns = []
for col in columns:
    if col not in dfm.columns:
        emptyColumns.append(col)
        dfm[col] = '' #fakedata[col]

In [25]:
dfm[dfm['cardFilter4Display'] != dfm['cardFilter4Display']]

,cardId,cardFilter1Tri,cardFilter2Tri,cardFilter3Tri,cardImage,cardSubtitle,cardFilter4Tri,cardFilter4Display,cardFilter0Tri,cardFilter5Display,...,cardFilter6Display,cardFilter6Tri,cardSection1Title,cardSection3Title,cardSection3Text,cardSection4Title,cardSection5Title,cardSection6Title,cardSection7Title,cardSection7Text
100,200,NaN,NaN,NaN,NaN,Digital first,NaN,NaN,no-card,NaN,...,,,,,,,,,,
101,300,NaN,NaN,NaN,NaN,Un regard sur l’avenir,NaN,NaN,no-card,NaN,...,,,,,,,,,,


In [26]:
chapitres = {
    'f-corporates': 'Grandes entreprises',
    'f-creatives': 'Créatifs',
    'f-do-gooders': 'Bienfaiteurs', 
    'f-enabler': 'Facilitateurs',
    'f-expats': 'Expats',
    'f-investors': 'Investisseurs',
    'f-researchers': 'Chercheurs',
    'f-serial-entrepreneurs': 'Serial entrepreneurs',
    'f-tech-leaders': 'Leaders',
    'f-unicorn-raisers': 'Éleveurs de licornes'
}

def getChapter(cardFilter):
    if cardFilter != cardFilter:
        return ''
    return chapitres[cardFilter]
dfm['cardFilter4Display'] = dfm['cardFilter4Tri'].apply(getChapter)

In [27]:
with open('editorial.txt', 'r') as f:
    edito = f.read()
for i, row in dfm[dfm['cardId'] == 200].iterrows():
    dfm.at[i,'cardTitre'] = 'Digital first'
    dfm.at[i, 'cardSection1Text'] = edito

In [28]:
dfm['cardFilter4DisplaySingular'] = dfm['cardFilter4Display'].apply(lambda x: x.lower()) #x.strip('s'))

In [29]:
jsondata = dfm.to_json(force_ascii=False, orient='records')

data['cards'] = json.loads(jsondata)

with open('/Users/rnp/Sites/digital-shapers/json/data.json', 'w', encoding='utf-8') as jsonfile:
    jsonfile.write(json.dumps(data, ensure_ascii=False, indent=4, sort_keys=True))

In [134]:
df

,cardId,Image,New_x,Age,cardFilter1Tri,cardFilter2Tri,cardFilter3Tri,Image_URL,cardImage,Name,...,cardSection5Text,cardSection6Text,isNew,Text,parameter,cardSection1Text,cardSection2Text,cardInfo1,cardInfo2,cardTitre
0,1,img/2018/Marco_Abele_Portrait.jpg,New,49 ans,f-plus-40,f-alemanique,f-homme,Marco_Abele_Portrait.jpg,img/2018/Marco_Abele_Portrait.jpg,Marco Abele,...,NaN,https://www.linkedin.com/in/marcoabele,is-new,De responsable numérique au CS à entrepreneur ...,marco-abele,De responsable numérique au CS à entrepreneur ...,,"Fondateur et CEO de Tend, Zoug",49 ans,Marco Abele
1,2,img/2018/Pedro_Bados.jpg,New,38 ans,f-moins-40,f-romand,f-homme,Pedro_Bados.jpg,img/2018/Pedro_Bados.jpg,Pedro Bados,...,Comment la société lausannoise Nexthink a levé...,https://ch.linkedin.com/in/pbados,is-new,«2017 a été une année incroyable. Nous avons g...,pedro-bados,«2017 a été une année incroyable. Nous avons g...,,"CEO et cofondateur de Nexthink, Lausanne",38 ans,Pedro Bados
2,3,img/2018/Nick_Beglinger_portrait.jpg,New,48 ans,f-plus-40,f-alemanique,f-homme,Nick_Beglinger_portrait.jpg,img/2018/Nick_Beglinger_portrait.jpg,Nick Beglinger,...,NaN,https://ch.linkedin.com/in/nick-beglinger-84a23b,is-new,"Jusqu’en 2016, Nick Beglinger a été président ...",nick-beglinger,"Jusqu’en 2016, Nick Beglinger a été président ...",,"Cofondateur et CEO de Cleantech21, Zurich",48 ans,Nick Beglinger
3,4,img/2018/Marc_P_Bernegger_Wikipedia.jpg,,39 ans,f-moins-40,f-alemanique,f-homme,Marc_P_Bernegger_Wikipedia.jpg,img/2018/Marc_P_Bernegger_Wikipedia.jpg,Marc Bernegger,...,«Apple pourrait ainsi devenir un acteur majeur...,https://ch.linkedin.com/in/marcpbernegger,not-new,"Spécialiste des fintech, Marc Bernegger se con...",marc-bernegger,"Spécialiste des fintech, Marc Bernegger se con...",,"Entrepreneur en fintech et investisseur, Zurich",39 ans,Marc Bernegger
4,5,img/2018/Sonja_Betschart_Portrait.jpg,New,48 ans,f-plus-40,f-romand,f-femme,Sonja_Betschart_Portrait.jpg,img/2018/Sonja_Betschart_Portrait.jpg,Sonja Betschart,...,L’Américain WeRobotics s’installe à Genève|htt...,https://ch.linkedin.com/in/sonjabetschart,is-new,"Passion, inspiration, innovation, ce sont les ...",sonja-betschart,"Passion, inspiration, innovation, ce sont les ...","Depuis deux ans et demi, Sonja Betschart trava...","Cofondatrice de WeRobotics, Genève",48 ans,Sonja Betschart
5,6,img/2018/Lucas_Betschart_Portrait.jpg,New,26 ans,f-moins-40,f-alemanique,f-homme,Lucas_Betschart_Portrait.jpg,img/2018/Lucas_Betschart_Portrait.jpg,Lucas Betschart,...,NaN,https://ch.linkedin.com/in/lucasbetschart,is-new,"Entre économistes, la blockchain et les monnai...",lucas-betschart,"Entre économistes, la blockchain et les monnai...",,Président de la Bitcoin Association Switzerlan...,26 ans,Lucas Betschart
6,7,img/2018/Jurgi_Camblong_Portrait.jpg,NaN,40 ans,f-plus-40,f-romand,f-homme,Jurgi_Camblong_Portrait.jpg,img/2018/Jurgi_Camblong_Portrait.jpg,Jurgi Camblong,...,«Le fondateur de Sophia Genetics nommé entrepr...,https://www.linkedin.com/in/jurgicamblong/,not-new,Jurgi Camblong parle d’une «révolution» de la ...,jurgi-camblong,Jurgi Camblong parle d’une «révolution» de la ...,,"Cofondateur et CEO de Sophia Genetics, Genève",40 ans,Jurgi Camblong
7,8,img/2018/Nicoletta_Casanova_Portrait.jpg,New,47 ans,f-plus-40,f-italienne,f-femme,Nicoletta_Casanova_Portrait.jpg,img/2018/Nicoletta_Casanova_Portrait.jpg,Nicoletta Casanova,...,NaN,https://www.linkedin.com/in/nicoletta-casanova...,is-new,"Son premier emploi, elle le trouva dans une st...",nicoletta-casanova,"Son premier emploi, elle le trouva dans une st...",,"Cofondatrice et CEO FEMTOprint, Muzzano (TI)",47 ans,Nicoletta Casanova
8,9,img/2018/Filippo_Catalano_Portrait.jpg,New,46 ans,f-plus-40,f-romand,f-homme,Filippo_Catalano_Portrait.jpg,img/2018/Filippo_Catalano_Portrait.jpg,Filippo Catalano,...,NaN,https://www.linkedin.com/in/filippocatalano/,is-new,Sa description du poste laisse songeur: il doi...,filippo-catalano,Sa description du poste laisse songeur: il

In [51]:
d = dfm['cardFilter4Display'].value_counts().sort_index().index

In [52]:
v = dfm['cardFilter4Tri'].value_counts().sort_index().index

In [53]:
for i, j in zip(v, d):
    print('<div class="btn" data-filter=".{}">{}</div>'.format(i, j))

<div class="btn" data-filter=".Automatique">A compléter: catégorie</div>
<div class="btn" data-filter=".f-corporates">Corporates</div>
<div class="btn" data-filter=".f-creatives">Creatives</div>
<div class="btn" data-filter=".f-do-gooders">Do-Gooders</div>
<div class="btn" data-filter=".f-enabler">Enabler</div>
<div class="btn" data-filter=".f-expats">Expats</div>
<div class="btn" data-filter=".f-investors">Investors</div>
<div class="btn" data-filter=".f-researchers">Researchers</div>
<div class="btn" data-filter=".f-serial-entrepreneurs">Serial Entrepreneurs</div>
<div class="btn" data-filter=".f-tech-leaders">Tech Leaders</div>
<div class="btn" data-filter=".f-unicorn-raisers">Unicorn raisers</div>


0                  Leaders
1     Éleveurs de licornes
2             Bienfaiteurs
3     Serial entrepreneurs
4             Bienfaiteurs
5            Facilitateurs
6     Éleveurs de licornes
7                  Leaders
8      Grandes entreprises
9                 Créatifs
10                  Expats
11              Chercheurs
12                 Leaders
13    Serial entrepreneurs
14                 Leaders
15            Bienfaiteurs
16              Chercheurs
17     Grandes entreprises
18            Bienfaiteurs
19           Facilitateurs
20              Chercheurs
21     Grandes entreprises
22            Bienfaiteurs
23    Éleveurs de licornes
24           Facilitateurs
25                Créatifs
26                Créatifs
27                  Expats
28              Chercheurs
29           Facilitateurs
              ...         
69                Créatifs
70                  Expats
71            Bienfaiteurs
72              Chercheurs
73                  Expats
74    Éleveurs de licornes
7

In [237]:
dxx = pd.DataFrame([['f-corporates', 'Grandes entreprises'],
['f-creatives', 'Créatifs'],
['f-do-gooders', 'Bienfaiteurs'], 
['f-enabler', 'Facilitateurs'],
['f-expats', 'Expats'],
['f-investors', 'Investisseurs'],
['f-researchers', 'Chercheurs'],
['f-serial-entrepreneurs', 'Serial entrepreneurs'],
['f-tech-leaders', 'Leaders'],
['f-unicorn-raisers', 'Éleveurs de licornes']])


In [243]:
dxx.sort_values(1, inplace=True)

In [245]:
v = dxx[0].tolist()
d = dxx[1].tolist()
for i, j in zip(v, d):
    print('<div class="btn" data-filter=".{}">{}</div>'.format(i, j))

<div class="btn" data-filter=".f-do-gooders">Bienfaiteurs</div>
<div class="btn" data-filter=".f-researchers">Chercheurs</div>
<div class="btn" data-filter=".f-creatives">Créatifs</div>
<div class="btn" data-filter=".f-expats">Expats</div>
<div class="btn" data-filter=".f-enabler">Facilitateurs</div>
<div class="btn" data-filter=".f-corporates">Grandes entreprises</div>
<div class="btn" data-filter=".f-investors">Investisseurs</div>
<div class="btn" data-filter=".f-tech-leaders">Leaders</div>
<div class="btn" data-filter=".f-serial-entrepreneurs">Serial entrepreneurs</div>
<div class="btn" data-filter=".f-unicorn-raisers">Éleveurs de licornes</div>


In [220]:
v = sorted(list(chapitres.values()))
d = [] 

['Bienfaiteurs',
 'Chercheurs',
 'Créatifs',
 'Expats',
 'Facilitateurs',
 'Grandes entreprises',
 'Investisseurs',
 'Leaders',
 'Serial entrepreneurs',
 'Éleveurs de licornes']

In [152]:
dfm.to_csv('json_export.csv')